## About

This document shows how clarity-ext is intended to be used by showing how it's built bottom-up.

In [1]:
from clarity_ext.domain import *
from clarity_ext import ClaritySession

## Sessions and repositories

In [9]:
# Create a session and load some data from a current step:
session = ClaritySession.create("24-3643")

# To get data from the API, we create a repository that uses this session object.
# Since we always fetch data through repository objects, mocking the data is simple
# (you only need to mock the repo objects)
from clarity_ext.repository import StepRepository
step = StepRepository(session)

# We can now fetch some data from the REST endpoint:
artifact_pairs = step.all_artifacts()
for inp, outp in artifact_pairs:
    print inp, "=>", outp

EdvardProv63 (ENG54A32) => 92-7103
EdvardProv63 (ENG54A32) => EdvardProv63 (ENG54A32)
EdvardProv62 (ENG54A31) => 92-7103
EdvardProv62 (ENG54A31) => EdvardProv62 (ENG54A31)
EdvardProv61 (ENG54A30) => EdvardProv61 (ENG54A30)
EdvardProv61 (ENG54A30) => 92-7103
EdvardProv60 (ENG54A29) => EdvardProv60 (ENG54A29)
EdvardProv60 (ENG54A29) => 92-7103


## Services

In [17]:
# The repository does only do simple mapping. The idea behind using it is that
# it will be the only thing you need to mock in order to unit test "service" objects.
# These objects do some further mapping or logic on the data and are the classes that
# will eventually be used.

# Let's say you want to get only the analytes in the current step. That's handled
# by the ArtifactService:
from clarity_ext.service import ArtifactService
svc = ArtifactService(step)
analytes = svc.all_analyte_pairs()
for analyte in analytes:
    print analyte.input_artifact, "=>", analyte.output_artifact

EdvardProv63 (ENG54A32) => EdvardProv63 (ENG54A32) False
EdvardProv62 (ENG54A31) => EdvardProv62 (ENG54A31) False
EdvardProv61 (ENG54A30) => EdvardProv61 (ENG54A30) False
EdvardProv60 (ENG54A29) => EdvardProv60 (ENG54A29) False


### Domain objects
Services don't return REST resources, they've always mapped those to "domain objects". These are objects that are supposed to be easier to use, as they've got related objects on them as fields or lazily evaluated properties. An example of how these objects are easier to use is that UDFs are fetched directly on the object.

In [21]:
print "Types: {}".format(set([type(i.input_artifact) for i in analytes]))

# We can fetch UDFs directly on the Analyte object, if it has been mapped accordingly. Note that
# these are currently semi-hardcoded fields, but they will be more flexible in a later version:
print analytes[0].input_artifact.concentration

Types: set([<class 'clarity_ext.domain.analyte.Analyte'>])
134.0


### Using services, examples:

In [27]:
# Now, you might also only need a list of all unique output containers. They
# are already properties on the Analyte domain object
print "Input:", svc.all_input_containers()
print "Output:", svc.all_output_containers()

Input: [27-629, 27-628]
Output: [27-631, 27-630]


## Unit testing services

In [31]:
# In order to unit test the logic in the system, only services (and utilities) need to be tested
# The data comes from "repository" classes, which can be mocked. They need to be integration tested against the
# server.

# TODO: Examples

# ExtensionContext

In [41]:
# When running an extension, the extension developer gets a "context" object to work with.
# This object has all necessary services wrapped in it

# The following code has already been executed by the extension runner:
from clarity_ext.extensions import ExtensionContext
context = ExtensionContext.create("24-3643")

# Now, the user can access the most common features directly, as lazy properties:
for container in context.output_containers:
    print container

27-631
27-630
